In [1]:
import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy

import tensorflow as tf
import tensorlayer as tl
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
from utils import *
from config import config, log_config

/home/hoppe/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
checkpoint_dir = "checkpoint"
save_dir = 'sample_dir'
path = '/home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4'

In [3]:
valid_lr_img_list = sorted(tl.files.load_file_list(path=path, regx='.*.png', printable=False))

valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, \
                        path=path, n_threads=32)

[TL] read 32 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 64 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 96 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 100 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4


In [4]:
valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, \
                        path=path, n_threads=32)

[TL] read 32 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 64 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 96 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 100 from /home/hoppe/Code/726/srgan-master/test_img/DIV2K_valid_LR_bicubic/X4


In [5]:
imid = 50  # 0: 企鹅  81: 蝴蝶 53: 鸟  64: 古堡
valid_lr_img = valid_lr_imgs[imid]
valid_lr_img = (valid_lr_img / 127.5) - 1

In [6]:
t_image = tf.placeholder('float32', [1, None, None, 3], name='input_image')
net_g = SRGAN_g(t_image, is_train=False, reuse=False)

[TL] InputLayer  SRGAN_g/in: (1, ?, ?, 3)
[TL] Conv2dLayer SRGAN_g/n64s1/c: shape:(3, 3, 3, 64) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] Conv2dLayer SRGAN_g/n64s1/c1/0: shape:(3, 3, 64, 64) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay:0.900000 epsilon:0.000010 act:relu is_train:False
[TL] Conv2dLayer SRGAN_g/n64s1/c2/0: shape:(3, 3, 64, 64) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay:0.900000 epsilon:0.000010 act:identity is_train:False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size:(1, ?, ?, 64) fn:add
[TL] Conv2dLayer SRGAN_g/n64s1/c1/1: shape:(3, 3, 64, 64) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay:0.900000 epsilon:0.000010 act:relu is_train:False
[TL] Conv2dLayer SRGAN_g/n64s1/c2/1: shape:(3, 3, 64, 64) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer SRGAN_g/n64s1/b2/1: decay:0.900000 epsilon:0.000010 act:identity is_train:F

[TL] SubpixelConv2d  pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2dLayer SRGAN_g/out: shape:(1, 1, 64, 3) strides:(1, 1, 1, 1) pad:SAME act:tanh


In [7]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_srgan.npz', network=net_g)

[TL] [*] Load checkpoint/g_srgan.npz SUCCESS!


In [8]:
out = sess.run(net_g.outputs, {t_image: [valid_lr_img]})

ResourceExhaustedError: OOM when allocating tensor with shape[256,64,3,3]
	 [[Node: SRGAN_g/n256s1/2/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](SRGAN_g/pixelshufflerx2/1/Relu, SRGAN_g/n256s1/2/W_conv2d/read)]]
	 [[Node: SRGAN_g/out/Tanh/_349 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_918_SRGAN_g/out/Tanh", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'SRGAN_g/n256s1/2/Conv2D', defined at:
  File "/home/hoppe/anaconda3/envs/tensorflow_env/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hoppe/anaconda3/envs/tensorflow_env/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hoppe/a

In [20]:
from PIL import Image
import numpy as np

img = Image.fromarray(valid_lr_imgs[50], 'RGB')
img.save('my.png')
img.show()